In [1]:
import json
import requests
import re,string,timeit
import time
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np

Setup your parameters
Request URL
https://api.domain.com.au/v2/suburbPerformanceStatistics/NSW/Castle%20Hill/2154?propertyCategory=house&bedrooms=3&periodSize=quarters&startingPeriodRelativeToCurrent=40&totalPeriods=40


Provide your client credentials as per your Domain developer account.

Required: client_id and client_secret

Make a POST request to receive token.

In [2]:
load_dotenv()
client_id =os.getenv("domain_client_id")
client_secret = os.getenv("domain_client_oauth_secrets")



In [3]:
#to authenticate client credentials
# Make a POST request to the https://auth.domain.com.au/v2/connect/token endpoint using your client_id and client_secret, along with the list of scopes that you require. 
auth_url2 = 'https://auth.domain.com.au/v1/connect/token'

In [4]:

# https://developer.domain.com.au/docs/v2/apis/pkg_properties_locations/references/suburbperformance_get_bynamedsuburb
# GET /v2/suburbPerformanceStatistics/{state}/{suburb}/{postcode}

In [5]:
#Scope and Permissions
#The following scopes may be used when calling this endpoint
#https://developer.domain.com.au/docs/v1/apis/pkg_properties_locations/references/properties_get:

scopes = ['api_suburbperformance_read']

In [6]:
url_endpoint2 = 'https://api.domain.com.au/v2/suburbPerformanceStatistics/'

In [ ]:
# TEST API : to define suburb details example for 1 suburb only ( this suburb has been tested on livebrowser)
# state='NSW'
# suburb="Chatswood"
# postcode='2067'
# # property_category = 'house'
# period_size = 'quarters'
# period_length = 40
# total_periods= 40
# url_detailed=f'https://api.domain.com.au/v2/suburbPerformanceStatistics/{state}/{suburb}/{postcode}/'


# url_detailed = 'https://api.domain.com.au/v2/suburbPerformanceStatistics/{}/{}/{}?propertyCategory={}&bedrooms={}&periodSize={}&startingPeriodRelativeToCurrent={}&totalPeriods={}'.format(state,suburb,postcode,propertyCategory,bedrooms,periodSize,startingPeriodRelativeToCurrent,totalPeriods)

In [7]:

df=pd.read_csv("./Resources/nsw_data.csv",index_col="Suburb")


In [8]:
df


,Postcode
Suburb,
Sydney,1001
Sydney,1002
Sydney,1003
Sydney,1004
Sydney,1005
Sydney,1006
Sydney,1007
Sydney,1008
Sydney,1009


In [9]:
#select suburbs starts from 2000 Sydney to eliminate postcodes in our df which are PO BOX
mask1=df['Postcode']>1999
list_suburbs=df[mask1]
# We, humans are biased, so let the Algorithm make the choice
list_suburbs=list_suburbs.loc[np.random.choice(list_suburbs.index,100,replace=False)]
list_suburbs=list_suburbs.to_dict()["Postcode"]
list_suburbs
# len(list_suburbs)

{'Hydes Creek': 2454,
 'Gillieston Heights': 2321,
 'Point Piper': 2027,
 'Yarra': 2580,
 'Telegraph Point': 2441,
 'Paxton': 2325,
 'Wiaborough': 2580,
 'Coolah': 2843,
 'Mortdale': 2223,
 'Fitzgeralds Valley': 2795,
 'Aldavilla': 2440,
 'Broadway': 2581,
 'Pleasant Hills': 2658,
 'Queanbeyan East': 2620,
 'Warringah Mall': 2100,
 'Brungle Creek': 2722,
 'Caloola': 2795,
 'Wimbledon': 2795,
 'Bendemeer': 2355,
 'Plumpton': 2761,
 'Possum Creek': 2479,
 'Forest Grove': 2795,
 'Salisbury': 2420,
 'Eastern Suburbs Mc': 2004,
 'Manly Vale': 2093,
 'Tullera': 2480,
 'Mount Hutton': 2290,
 'Dicks Hill': 2443,
 'Penrose': 2579,
 'Dawes Point': 2000,
 'Brewongle': 2795,
 'Wamoon': 2705,
 'Cooperabung': 2441,
 'Coocooboonah': 2380,
 'Canbelego': 2835,
 'Moonbria': 2710,
 'Yetholme': 2795,
 'Lemington': 2330,
 'Mount Mcdonald': 2793,
 'Bow Bowing': 2566,
 'Yarravel': 2440,
 'Badgerys Creek': 2555,
 'Campbelltown': 2560,
 'Cudgel': 2700,
 'Berrambool': 2548,
 'Bungabbee': 2480,
 'Koraleigh': 273

In [10]:
final_list_suburbs = []

In [22]:
#housing data with bedrooms, houseCategories, timeperiods length
def get_property_info(suburb,postcode):
    response = requests.post(auth_url2,
        data = {
        'client_id':client_id,
        'client_secret':client_secret,
        'grant_type':'client_credentials',
        'scope': scopes,
        'Content-Type':'text/json'}
        )
    json_res = response.json()
    access_token=json_res["access_token"]
    print(access_token)
    auth = {'Authorization':'Bearer '+access_token}
    
    url_detailed=f'https://api.domain.com.au/v2/suburbPerformanceStatistics/NSW/{suburb}/{postcode}/?totalPeriods=40'

    res2 = requests.get(url_detailed,headers = auth)
    r2=res2.json()
    json_object =json.dumps(r2,indent=4)
    json.dumps(r2,indent=4)
            
    if "header"  in json.loads(json_object).keys(): 

        with open("./resources/{}_{}.json".format(postcode,suburb),"w") as outfile:
            outfile.write(json_object)   
    else:

        print('Failure in',suburb, postcode,e,access_token)






In [24]:
#  final_list_suburbs.append(suburb)
for suburb,postcode in list_suburbs.items():
    try:
        get_property_info(suburb,postcode)
        print('Success', suburb, postcode)
        final_list_suburbs.append(suburb)

    except Exception as e:
       
        print('Failure in',suburb, postcode,e)
        pass
    
   

a876ac6a8bea095e6616a406151fa112
Success Hydes Creek 2454
ed8353605509384b09fdbac21b3270be
Success Gillieston Heights 2321
5b0b47f3861a721ec2df8d432aeac68b
Success Point Piper 2027
c4205942fe04029e5ea95e5ec4bfc914
Success Yarra 2580
42f47a93da2de6892c078d92eb235f3e
Success Telegraph Point 2441
ff61a87117dc0e352b4c48a0cf0babbd
Success Paxton 2325
740ee4546ccfd230915a00e0362180e2
Failure in Wiaborough 2580 Expecting value: line 1 column 1 (char 0)
79db0f3000849f1af2d46fa4338f31c7
Success Coolah 2843
6cad7322a48c6e1f0d5b1d0595384bb9
Success Mortdale 2223
e075966a8d8355d5dfa2d46d352b9e6a
Success Fitzgeralds Valley 2795
17953361ec373fb2d96a710a093c8fb0
Success Aldavilla 2440
7058bc1f75f3fc81206817e070e77536
Success Broadway 2581
1ef2dbca6652b83722f80ddb27d33e6b
Success Pleasant Hills 2658
776503b08940b7db34a15698e8c2d744
Success Queanbeyan East 2620
079776fef8fa1b1c6a5842632e33ca2f
Failure in Warringah Mall 2100 Expecting value: line 1 column 1 (char 0)
5dbbae7d3d350237413abeab50ac443c
Succ

In [13]:
len(final_list_suburbs)

83

In [14]:
#to open json file saved in previous step & convert to dataframe & combine all dataframe into a master dataframe
df = pd.DataFrame()
frames= []
for suburb in final_list_suburbs: 
    with open("./resources/{}_{}.json".format(list_suburbs[suburb],suburb),"r") as sub_housing_data:
        sub_housing_data=json.load(sub_housing_data)
        suburb_name=sub_housing_data['header']['suburb']
        
    for x in range (40):
        
        headers_dict1=sub_housing_data['series']['seriesInfo'][x]['values']
        headers_dict1["year"]=sub_housing_data['series']['seriesInfo'][x]["year"]
        headers_dict1["month"]=sub_housing_data['series']['seriesInfo'][x]["month"]
        sub_df1 = pd.DataFrame(headers_dict1.values(),index=headers_dict1.keys(),columns=[suburb_name]).T
      
        sub_df1 = sub_df1.reset_index().drop(columns=["auctionNumberWithdrawn","daysOnMarket","discountPercentage",
                                                      "5thPercentileSoldPrice","25thPercentileSoldPrice",
                                                      "75thPercentileSoldPrice","95thPercentileSoldPrice",
                                                      "numberRentListing","highestRentListingPrice","lowestRentListingPrice"]).rename(columns={"index":"suburb"})
        frames.append(sub_df1)
df = pd.concat(frames)
        

    

In [15]:
df.head(10)

,suburb,medianSoldPrice,numberSold,highestSoldPrice,lowestSoldPrice,medianSaleListingPrice,numberSaleListing,highestSaleListingPrice,lowestSaleListingPrice,auctionNumberAuctioned,auctionNumberSold,medianRentListingPrice,year,month
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,9.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,12.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.0,3.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.0,6.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.0,9.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.0,12.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,3.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,6.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,9.0
0,Hydes Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,12.0


In [25]:
df.columns

Index(['year', 'month', 'suburb', 'medianSoldPrice', 'numberSold',
       'highestSoldPrice', 'lowestSoldPrice', 'medianSaleListingPrice',
       'numberSaleListing', 'highestSaleListingPrice',
       'lowestSaleListingPrice', 'auctionNumberAuctioned', 'auctionNumberSold',
       'medianRentListingPrice'],
      dtype='object')

In [26]:
df = df[['year', 'month','suburb', 'medianSoldPrice', 'numberSold', 'highestSoldPrice',
       'lowestSoldPrice', 'medianSaleListingPrice', 'numberSaleListing',
       'highestSaleListingPrice', 'lowestSaleListingPrice',
       'auctionNumberAuctioned', 'auctionNumberSold', 'medianRentListingPrice',
       ]]

In [27]:
df.sample(10)

,year,month,suburb,medianSoldPrice,numberSold,highestSoldPrice,lowestSoldPrice,medianSaleListingPrice,numberSaleListing,highestSaleListingPrice,lowestSaleListingPrice,auctionNumberAuctioned,auctionNumberSold,medianRentListingPrice
0,2018.0,3.0,Towrang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2013.0,3.0,Baryulgil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021.0,6.0,Manly Vale,2805000.0,11.0,3825000.0,1525000.0,NaN,8.0,2750000.0,2200000.0,6.0,6.0,1100.0
0,2011.0,9.0,Clydesdale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2015.0,9.0,Cudgegong,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2013.0,12.0,Timbumburi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021.0,6.0,Hat Head,NaN,NaN,NaN,NaN,NaN,2.0,1395000.0,750000.0,NaN,NaN,NaN
0,2011.0,12.0,Koraleigh,NaN,NaN,NaN,NaN,NaN,1.0,120000.0,120000.0,NaN,NaN,NaN
0,2020.0,9.0,Clydesdale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2016.0,6.0,Stroud Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df.to_csv('data_V2_25082021.csv',index=False)